# Load required packages

In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# Load data

In [2]:
path = "/Users/tian/Downloads/lish-moa/"

In [3]:
train_df = pd.read_csv(path+'train_features.csv',index_col=['sig_id'])
train_target_df = pd.read_csv(path+'train_targets_scored.csv',index_col=['sig_id'])

In [4]:
test_df = pd.read_csv(path+'test_features.csv',index_col=['sig_id'])

In [5]:
train_df.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [6]:
train_target_df.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000779bfc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_000a6266a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_0015fd391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_001626bd3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
for i in range(3):
    print(f"{train_df.iloc[:,i].value_counts()}'\n'")

trt_cp         21948
ctl_vehicle     1866
Name: cp_type, dtype: int64'
'
48    8250
72    7792
24    7772
Name: cp_time, dtype: int64'
'
D1    12147
D2    11667
Name: cp_dose, dtype: int64'
'


# Preprocessing

In [8]:
train_df.isna().sum().sum()

0

In [137]:
def preprocess(df):
    df = df.copy()
    df['cp_type'] = df['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
    df['cp_dose'] = df['cp_dose'].map({'D1':0, 'D2':1})
    df['cp_time'] = df['cp_time'].map({24:0,48:1,72:2})
    return df

train = preprocess(train_df)
test = preprocess(test_df)

In [138]:
train_target_df = train_target_df.loc[train['cp_type']==0]
train = train.loc[train['cp_type']==0]
train.drop(columns=['cp_type'], inplace=True)

test = test.loc[test['cp_type']==0]
test.drop(columns=['cp_type'], inplace=True)

In [99]:
# test one dependent variable
# train = train.merge(train_target_df.iloc[:,0],left_index=True,right_index=True,how='inner')
# le = LabelEncoder()
# #le.categories_
# train['cp_time'] = le.fit_transform(train['cp_time']) 
# test['cp_time'] = le.transform(test['cp_time'])

In [142]:
train = train.merge(train_target_df, how='inner', left_index=True, right_index=True)
train.head()

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0,0,0,0,0,0,0,0,0,0
id_000779bfc,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,0,0,0,0,0,0,0,0,0,0
id_000a6266a,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,0,0,0,0,0,0,0,0,0,0
id_0015fd391,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,0,0,0,0,0,0,0,0,0,0
id_001626bd3,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0,0,0,0,0,0,0,0,0,0


In [171]:
shuffle_df = train.sample(frac=1, random_state=1)
ratio = int(len(shuffle_df)*0.75)
Xtrain = shuffle_df.iloc[:ratio]
Xtest = shuffle_df.iloc[ratio:]
print(f"{Xtrain.shape}, {Xtest.shape}")

(16461, 1080), (5487, 1080)


In [172]:
ytrain = Xtrain.loc[:,'5-alpha_reductase_inhibitor':]
ytest = Xtest.loc[:,'5-alpha_reductase_inhibitor':]
Xtrain = Xtrain.loc[:,:'5-alpha_reductase_inhibitor'].iloc[:,:-1]
Xtest = Xtest.loc[:,:'5-alpha_reductase_inhibitor'].iloc[:,:-1]

In [173]:
print(f'{Xtrain.shape}\n {Xtest.shape}\n {ytrain.shape}\n {ytest.shape}')

(16461, 874)
 (5487, 874)
 (16461, 206)
 (5487, 206)


In [175]:
Scaler = MinMaxScaler()
Xtrain_scaled = Scaler.fit_transform(Xtrain.iloc[:,2:])
Xtest_scaled = Scaler.transform(Xtest.iloc[:,2:])

In [176]:
Xtrain_scaled = np.concatenate([Xtrain.iloc[:,:2].values, Xtrain_scaled], axis = 1)
Xtest_scaled = np.concatenate([Xtest.iloc[:,:2].values, Xtest_scaled], axis = 1)

In [180]:
Xtrain_scaled

array([[1.        , 0.        , 0.33936054, ..., 0.7314299 , 0.71596369,
        0.63882651],
       [1.        , 0.        , 0.78398762, ..., 0.27715877, 0.1973915 ,
        0.03462514],
       [0.        , 1.        , 0.37186876, ..., 0.79695915, 0.75464877,
        0.70156465],
       ...,
       [1.        , 0.        , 0.30463482, ..., 0.73126741, 0.74662497,
        0.70671496],
       [2.        , 1.        , 0.33308838, ..., 0.74079232, 0.74631988,
        0.72344803],
       [2.        , 1.        , 0.34013408, ..., 0.7598112 , 0.72194341,
        0.75406737]])

In [181]:
ytrain

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_ce5bda867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_9dd4cd684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_d21376056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_a83a4a865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
id_a9827661c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_06a4a1ba5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_e6da104d7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
id_07ed6ce4d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Baseline model

In [232]:
clf = RandomForestClassifier(max_features=8, max_depth=3, class_weight={0:0.08,1:0.92},random_state=0)
clf.fit(Xtrain_scaled, ytrain.iloc[:,3])

RandomForestClassifier(class_weight={0: 0.08, 1: 0.92}, max_depth=3,
                       max_features=8, random_state=0)

In [233]:
clf.predict_proba(Xtest_scaled)[np.argsort(clf.predict_proba(Xtest_scaled)[:,1])]

array([[0.99043762, 0.00956238],
       [0.99006679, 0.00993321],
       [0.98991331, 0.01008669],
       ...,
       [0.85790427, 0.14209573],
       [0.85106471, 0.14893529],
       [0.83898533, 0.16101467]])

In [234]:
clf.score(Xtest_scaled, ytest.iloc[:,3])

0.9919810461089849

In [235]:
44/5487

0.008018953891015127

In [236]:
confusion_matrix(clf.predict(Xtest_scaled), ytest.iloc[:,3])

array([[5443,   44],
       [   0,    0]])

In [211]:
cw = compute_class_weight('balanced', [0,1], ytrain.iloc[:,0])
cw/2.11

array([2.37154116e-01, 3.00054685e+02])

In [23]:
def rfclusters (cols, df):
    res = []
    for i in cols:
        clf = RandomForestClassifier(max_features=8,
                                     max_depth=3,
                                     n_estimators = 500,
                                     random_state=0)
        clf.fit(Xtrain_scaled, )
        clf.
    

5-alpha_reductase_inhibitor
11-beta-hsd1_inhibitor
acat_inhibitor
acetylcholine_receptor_agonist
acetylcholine_receptor_antagonist
acetylcholinesterase_inhibitor
adenosine_receptor_agonist
adenosine_receptor_antagonist
adenylyl_cyclase_activator
adrenergic_receptor_agonist
adrenergic_receptor_antagonist
akt_inhibitor
aldehyde_dehydrogenase_inhibitor
alk_inhibitor
ampk_activator
analgesic
androgen_receptor_agonist
androgen_receptor_antagonist
anesthetic_-_local
angiogenesis_inhibitor
angiotensin_receptor_antagonist
anti-inflammatory
antiarrhythmic
antibiotic
anticonvulsant
antifungal
antihistamine
antimalarial
antioxidant
antiprotozoal
antiviral
apoptosis_stimulant
aromatase_inhibitor
atm_kinase_inhibitor
atp-sensitive_potassium_channel_antagonist
atp_synthase_inhibitor
atpase_inhibitor
atr_kinase_inhibitor
aurora_kinase_inhibitor
autotaxin_inhibitor
bacterial_30s_ribosomal_subunit_inhibitor
bacterial_50s_ribosomal_subunit_inhibitor
bacterial_antifolate
bacterial_cell_wall_synthesis_inh

In [39]:
train_target_df[cols[2]]

sig_id
id_000644bb2    0
id_000779bfc    0
id_000a6266a    0
id_0015fd391    0
id_001626bd3    0
               ..
id_fff8c2444    0
id_fffb1ceed    0
id_fffb70c0c    0
id_fffcb9e7c    0
id_ffffdd77b    0
Name: acat_inhibitor, Length: 21948, dtype: int64